## 2. Perform KWIC Searches using `getout_of_text_3` on COCA, GloWbE, and EcoLexicon

- Author: [Etienne P Jacquot](mailto:etienne.jacquot@asc.upenn.edu) (11/14/2025)

### Corpora

Both COCA and GloWbE now use **consistent nested structure**:

- **`corpus_name='coca'`** (default): `{genre: {year_or_id: DataFrame}}`
  - Example: `coca['acad']['2009']` → DataFrame
  
- **`corpus_name='glowbe'`**: `{country_code: {file_id: DataFrame}}`
  - Example: `glowbe['us']['g19']` → DataFrame
  
- **`corpus_name='EcoLexicon'`**: 
  - Manual CSV downloads from Sketch-Engine EcoLexicon
________________

In [ ]:
!pip install -U getout_of_text_3 -qqq

## Run Imports

To use the latest code changes (including reading COCA & GloWbE offline licensed files from English-Corpora.org), import the module after installation: 

In [ ]:
import getout_of_text_3 as got3
import pandas as pd
import os
    
print("getout_of_text_3 version:",  got3.__version__)

## Read Local Corpus Data into Notebook

- the `corpus_name` parameter specifies which corpus format to use (`'coca'` or `'glowbe'`), as each have different directory structures and file naming conventions.

    > Note: The module will automatically fall back to the COCA formatting if an unrecognized corpus name is provided. If you are using a English-Corpora.org corpus that is not supported (i.e. have a different structure) please create an issue on GitHub bringing this to the attention of the maintainer

In [ ]:
# Define data directory of your local English-Corpora.org offline files
data_directory = '../../data/english-corpora.org/'

In [ ]:
# Load COCA corpus
coca_corpus = got3.read_corpus(dir_of_text_files='{}/coca/'.format(data_directory), 
                               corpus_name='coca')

In [ ]:
# Load GloWbE corpus
glowbe_corpus = got3.read_corpus(dir_of_text_files='{}/glowbe/'.format(data_directory), 
                                 corpus_name='glowbe')

In [ ]:
# Manually download CSVs from Sketch Engine and load into a format compatible with getout_of_text_3
ecolexicon_corpus = {}

for csv_file in [file for file in os.listdir('../../data/sketch-engine/') if file.endswith('.csv')]:
    
    # read your Sketch-Engine exported CSV file, skipping the first 4 rows of metadata
    df = pd.read_csv('../../data/sketch-engine/{}'.format(csv_file), skiprows=4)

    # add a 'text' column combining Left, KWIC, and Right context (i.e. the original concordance lines)
    df['text']= df['Left']+' '+df['KWIC']+' '+df['Right']

    # store the dataframe in the ecolexicon_corpus dictionary, using a placeholder genre value 'eco_text's
    ecolexicon_corpus[csv_file] = {'eco_text': df}

___________________
## Perform Word Count Totals for Each Corpus

This step is to verify that the corpora have been read in correctly by checking total word counts against known values from English-Corpora.org documentation.

- **COCA**: 1.1 billion words
- **GloWbE**: 1.9 billion words
- **EcoLexicon**: 23.1 million words (*ignoring given we do not have the full database locally*)

In [ ]:
# Calculate total word count across all COCA genres and subkeys
def count_words_in_text(text):
    """Count words in a text string."""
    if not isinstance(text, str):
        return 0
    return len(text.split())

#
def calculate_total_word_count(corpus):
    total_word_count = 0
    start_time=pd.Timestamp.now()
    for genre, subkeys in corpus.items():
        for subkey, dataframe in subkeys.items():
            if isinstance(dataframe, pd.DataFrame) and 'text' in dataframe.columns:
                # Count words in all text entries for this subkey
                subkey_word_count = dataframe['text'].apply(count_words_in_text).sum()
                total_word_count += subkey_word_count

    end_time=pd.Timestamp.now()
    elapsed_time = end_time - start_time
    print(f"⏱️ Elapsed time: {elapsed_time}")
    print(f"🎯 TOTAL GLOWBE CORPUS: {total_word_count:,} words")
    return total_word_count

In [ ]:
coca_word_count = calculate_total_word_count(coca_corpus)
glowbe_word_count = calculate_total_word_count(glowbe_corpus)
ecolexicon_word_count = 23100000

_____________________
## Running Query Searches Across English-Corpora `COCA`

- **Corpus of Contemporary American English (COCA)**
    - See the public corpus here: https://www.english-corpora.org/coca
    - Run for the terms:
        - `best system`
        - `best method`
        - `national system`
        - `industry standard`


In [ ]:
keyword='best system'
corpus_name='coca'
best_system_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
# drop empty results
best_system_kwic_coca = {k: v for k, v in best_system_kwic_coca.items() if v}
# get total count
best_system_total_kwic_coca = sum(len(v) for v in best_system_kwic_coca.values())


In [ ]:
keyword='best method'
corpus_name='coca'
best_method_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_coca = {k: v for k, v in best_method_kwic_coca.items() if v}
best_method_total_kwic_coca = sum(len(v) for v in best_method_kwic_coca.values())


In [ ]:
keyword='national system'
corpus_name='coca'
national_system_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_coca = {k: v for k, v in national_system_kwic_coca.items() if v}
national_system_total_kwic_coca = sum(len(v) for v in national_system_kwic_coca.values())


In [ ]:
keyword='industry standard'
corpus_name='coca'
industry_standard_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_coca = {k: v for k, v in industry_standard_kwic_coca.items() if v}
industry_standard_total_kwic_coca = sum(len(v) for v in industry_standard_kwic_coca.values())


_____________________
## Running Query Searches Across English-Corpora `GloWbE`

- **Corpus of Global Web-Based English (GloWbE)**
    - See the public corpus here: https://www.english-corpora.org/glowbe
    - Run for the terms:
        - `best system`
        - `best method`
        - `national system`
        - `industry standard`


In [ ]:
keyword='best system'
corpus_name='glowbe'
best_system_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_system_kwic_glowbe = {k: v for k, v in best_system_kwic_glowbe.items() if v}
best_system_total_kwic_glowbe = sum(len(v) for v in best_system_kwic_glowbe.values())

In [ ]:
keyword='best method'
corpus_name='glowbe'
best_method_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_glowbe = {k: v for k, v in best_method_kwic_glowbe.items() if v}
best_method_total_kwic_glowbe = sum(len(v) for v in best_method_kwic_glowbe.values())

In [ ]:
keyword='national system'
corpus_name='glowbe'
national_system_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_glowbe = {k: v for k, v in national_system_kwic_glowbe.items() if v}
national_system_total_kwic_glowbe = sum(len(v) for v in national_system_kwic_glowbe.values())


In [ ]:
keyword='industry standard'
corpus_name='glowbe'
industry_standard_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_glowbe = {k: v for k, v in industry_standard_kwic_glowbe.items() if v}
industry_standard_total_kwic_glowbe = sum(len(v) for v in industry_standard_kwic_glowbe.values())


_____________________
## Running Query Searches Across Sketch Engine `EcoLexicon-Corpus`

- See the public corpus here: https://www.sketchengine.eu/ecolexicon-corpus/
- Manually downloaded from Sketch Engine web interface as CSV files for the `EcoLexicon` corpus for the terms:
  - `best system`
  - `best method`
  - `national system`
  - `industry standard`


In [ ]:
keyword='best system'
corpus_name='ecolexicon'
best_system_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_system_kwic_ecolexicon = {k: v for k, v in best_system_kwic_ecolexicon.items() if v}
best_system_total_kwic_ecolexicon = sum(len(v) for v in best_system_kwic_ecolexicon.values())


In [ ]:
keyword='best method'
corpus_name='ecolexicon'
best_method_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_ecolexicon = {k: v for k, v in best_method_kwic_ecolexicon.items() if v}
best_method_total_kwic_ecolexicon = sum(len(v) for v in best_method_kwic_ecolexicon.values())

In [ ]:
keyword='national system'
corpus_name='ecolexicon'
national_system_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_ecolexicon = {k: v for k, v in national_system_kwic_ecolexicon.items() if v}
national_system_total_kwic_ecolexicon = sum(len(v) for v in national_system_kwic_ecolexicon.values())

In [ ]:
keyword='industry standard'
corpus_name='ecolexicon'
industry_standard_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_ecolexicon = {k: v for k, v in industry_standard_kwic_ecolexicon.items() if v}
industry_standard_total_kwic_ecolexicon = sum(len(v) for v in industry_standard_kwic_ecolexicon.values())

## Create Metadata DataFrame

Compile comprehensive statistics for KWIC results including:
- **Corpus**: Name of the corpus (COCA/GloWbE)
- **Total Genres**: Number of genres/categories in corpus
- **Total Word Count**: Total words in entire corpus
- **KWIC Genres**: Genres with keyword hits
- **KWIC Hits**: Total number of keyword occurrences
- **Hits per Million**: Normalized frequency (hits per 1M words)

In [ ]:
def create_kwic_metadata(corpus_name, keyword_name, corpus_dict, kwic_results, total_word_count):
    """
    Create metadata row for KWIC search results.
    
    Args:
        corpus_name: Name of corpus (e.g., 'COCA', 'GloWbE')
        corpus_dict: Full corpus dictionary
        kwic_results: KWIC search results dictionary
        total_word_count: Total words in corpus
    
    Returns:
        Dictionary with metadata statistics
    """
    # Count total genres in corpus
    total_genres = len(corpus_dict.keys())
    
    # Count genres with KWIC hits
    kwic_genres = len(kwic_results.keys())
    
    # Count total KWIC hits
    total_kwic_hits = sum(len(v) for v in kwic_results.values())
    
    # Calculate hits per million words
    hits_per_million = (total_kwic_hits / total_word_count) * 1_000_000 if total_word_count > 0 else 0
    
    return {
        'Corpus': corpus_name,
        'Keyword': keyword_name,
        'Total Genres': total_genres,
        'Total Word Count': total_word_count,
        'KWIC Genres': kwic_genres,
        'KWIC Hits': total_kwic_hits,
        'Hits per Million': round(hits_per_million, 2)
    }


In [ ]:
# Create metadata rows for both corpora
metadata_rows = []

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='best system',
    corpus_dict=coca_corpus,
    kwic_results=best_system_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='best system',
    corpus_dict=glowbe_corpus,
    kwic_results=best_system_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='best system',
    corpus_dict=ecolexicon_corpus,
    kwic_results=best_system_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='best method',
    corpus_dict=coca_corpus,
    kwic_results=best_method_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='best method',
    corpus_dict=glowbe_corpus,
    kwic_results=best_method_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='best method',
    corpus_dict=ecolexicon_corpus,
    kwic_results=best_method_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='national system',
    corpus_dict=coca_corpus,
    kwic_results=national_system_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='national system',
    corpus_dict=glowbe_corpus,
    kwic_results=national_system_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='national system',
    corpus_dict=ecolexicon_corpus,
    kwic_results=national_system_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='industry standard',
    corpus_dict=coca_corpus,
    kwic_results=industry_standard_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='industry standard',
    corpus_dict=glowbe_corpus,
    kwic_results=industry_standard_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='industry standard',
    corpus_dict=ecolexicon_corpus,
    kwic_results=industry_standard_kwic_ecolexicon,
    total_word_count=2310000
))


In [ ]:
import pandas as pd
metadata_df = pd.DataFrame(metadata_rows)
metadata_df.sort_values(by=['Corpus', 'Keyword'], inplace=True)
print(f"📊 Updated Metadata DataFrame created with {len(metadata_df)} rows")


In [ ]:
metadata_df['Total Word Count'] = metadata_df['Total Word Count'].astype(int)
metadata_df.style.format({
    'Total Word Count': '{:,}',
    'KWIC Hits': '{:,}',
    'Hits per Million': '{:.2f}'
})

In [ ]:
metadata_df_temp = metadata_df[['Corpus', 'Keyword', 'Total Word Count', 
                                'KWIC Hits', 'Hits per Million']]
metadata_df_temp.style.format({
    'Total Word Count': '{:,}',
    'KWIC Hits': '{:,}',
    'Hits per Million': '{:.2f}'
})

In [ ]:
# Create a summary showing corpus metadata (non-duplicated)
corpus_summary = metadata_df.groupby('Corpus').agg({
    'Total Genres': 'first',
    'Total Word Count': 'first'
}).reset_index()

print("Corpus Summary (Total Word Counts):")
display(corpus_summary)

print("\nKWIC Hits by Keyword and Corpus:")
pivot_hits = metadata_df.pivot(index='Keyword', columns='Corpus', values='KWIC Hits')
display(pivot_hits)


___________________________
## Export KWIC results and metadata file to JSON

In [ ]:
metadata_df.to_json('./exports/metadata_11142025_df.json')

In [ ]:

from pathlib import Path
import json
import numpy as np
import pandas as _pd

# ensure exports directory exists
exports_dir = Path('./exports')
exports_dir.mkdir(parents=True, exist_ok=True)

def _serialize(obj):
    """Recursively convert common non-JSON-serializable objects to JSON-friendly types."""
    # numpy scalars / arrays
    if isinstance(obj, (np.integer,)):
        return int(obj)
    if isinstance(obj, (np.floating,)):
        return float(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    # pandas objects (pd imported earlier in the notebook)
    try:
        if isinstance(obj, _pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, _pd.DataFrame):
            return obj.to_dict(orient='records')
        if isinstance(obj, _pd.Series):
            return obj.tolist()
    except Exception:
        pass
    # Builtins
    if isinstance(obj, dict):
        return {k: _serialize(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [_serialize(v) for v in obj]
    if isinstance(obj, tuple):
        return [_serialize(v) for v in obj]
    # Fallback to string for unknown types
    return obj

# Serialize and dump KWIC results for each keyword and corpus combination
# COCA exports
kwic_coca_best_system = _serialize(best_system_kwic_coca)
kwic_coca_best_method = _serialize(best_method_kwic_coca)
kwic_coca_national_system = _serialize(national_system_kwic_coca)
kwic_coca_industry_standard = _serialize(industry_standard_kwic_coca)

with open(exports_dir / 'kwic_coca_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_industry_standard, f, ensure_ascii=False, indent=2)

# GloWbE exports
kwic_glowbe_best_system = _serialize(best_system_kwic_glowbe)
kwic_glowbe_best_method = _serialize(best_method_kwic_glowbe)
kwic_glowbe_national_system = _serialize(national_system_kwic_glowbe)
kwic_glowbe_industry_standard = _serialize(industry_standard_kwic_glowbe)

with open(exports_dir / 'kwic_glowbe_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_industry_standard, f, ensure_ascii=False, indent=2)

# EcoLexicon exports
kwic_ecolexicon_best_system = _serialize(best_system_kwic_ecolexicon)
kwic_ecolexicon_best_method = _serialize(best_method_kwic_ecolexicon)
kwic_ecolexicon_national_system = _serialize(national_system_kwic_ecolexicon)
kwic_ecolexicon_industry_standard = _serialize(industry_standard_kwic_ecolexicon)

with open(exports_dir / 'kwic_ecolexicon_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_industry_standard, f, ensure_ascii=False, indent=2)

print(f"✅ Written 12 KWIC JSON files to {exports_dir.resolve()}:")
print("   COCA: best_system, best_method, national_system, industry_standard")
print("   GloWbE: best_system, best_method, national_system, industry_standard")
print("   EcoLexicon: best_system, best_method, national_system, industry_standard")

____________________________

### Proceed for Minimal Coding Approach

- concordance line annotation
- use the `annotator_app/app.py` to annotate concordance lines interactively